In [ ]:
%pip install pyalex
%pip install leidenalg
%pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 18.8 MB/s eta 0:00:00


In [17]:
from pyalex import Works, Authors, Sources, Institutions, Concepts, Funders
import pyalex,ast,pandas as pd, networkx as nx, numpy as np, matplotlib.pyplot as plt
from tqdm import tqdm
import matplotlib.patheffects as patheffects
import random,time,itertools,tqdm,collections
from sklearn.feature_extraction.text import TfidfVectorizer
import re
flatten = lambda x: list(itertools.chain.from_iterable(x))

pyalex.config.email = "miyabe-hiroki806@g.ecc.u-tokyo.ac.jp" # ここにあなたのメールアドレスを入力してください

In [23]:
def getAll(pyalexObj):
    pager = pyalexObj.paginate(per_page=200,n_max=None)
    arr = []
    for page in tqdm.tqdm(pager):
        arr += page
    return arr

def get_journal_name_original(citing):
    '''
    citnig(列)：['primary_location']列しか取れない

    '''
    try:
        return citing['source']['display_name']
    except:
        return None

def get_topic(topic_raw,class_):
    '''
    topic()：['primary_location']列しか取れない
    class(str):domain,field,subfieldから選ぶ
    '''
    if class_ == "domain":
        try:
          return topic_raw['domain']["display_name"]

        except:
          return None

    elif class_ == "field":
        try:
          return topic_raw['field']["display_name"]

        except:
          return None

    elif class_ == "subfield":

        try:
         return topic_raw['subfield']["display_name"]

        except:
          return None



def remove_parentheses(text):
    """
    文字列から()で囲まれた部分を含む文字列を削除し、
    ()の前に半角スペースがある場合、そのスペースも削除する関数

    Parameters:
        text (str): 対象の文字列

    Returns:
        str: ()で囲まれた部分が削除された文字列
    """
    return re.sub(r'\s*\([^()]*\)', '', text)




In [19]:
cols = ["id","title",'abstract_inverted_index',"publication_year","primary_topic",'primary_location']

In [24]:
#ジャーナルの指定
source_ids = [
    "https://openalex.org/sources/s202144432", # Science, technology & society
    "https://openalex.org/sources/s2181421",   # Science, technology & human values
    "https://openalex.org/sources/s68632876",  # Social studies of science
    "https://openalex.org/sources/s104981805", # East Asian science, technology and society
    "https://openalex.org/sources/s124907306", # Public understanding of science
    "https://openalex.org/sources/s30803906",  # JCOM, journal of science communication
    "https://openalex.org/sources/s4210212669",# science communication
    "https://openalex.org/sources/s190143089", # Science as culture
    "https://openalex.org/sources/s16793705",  # Research Evaluation
    "https://openalex.org/sources/s9731383",   # Research policy(4800)
    "https://openalex.org/sources/s68632876",  # Science and public policy,
    "https://openalex.org/sources/s148561398"  # scientometorics
]

#キーワードの指定
keyword = '"covid"|COVID-19|"corona"|"sars-cov-2"'
THRESHOLD = 0
pyalexObj = Works().filter(locations={"source":{"id":"|".join(source_ids)}},title_and_abstract={"search":keyword}, publication_year=f">{2018}", cited_by_count=f">{THRESHOLD-1}",has_references="true").select(cols)

###

field_count = pyalexObj.count()
print(field_count)
#https://note.nkmk.me/python-pandas-duplicated-drop-duplicates/
#引数subsetで判定する列を指定できる。
#つまり同じ論文を削除している
works =  pd.DataFrame(getAll(pyalexObj)).drop_duplicates(subset=['id'])
print(f"取得した論文数/分野内の論文数：{len(works)}/{field_count} ({round(len(works)/field_count*100,2)}%)" )

208


4it [00:03,  1.17it/s]

取得した論文数/分野内の論文数：208/208 (100.0%)


In [ ]:
works.head()

In [25]:
works["primary_location"] = works["primary_location"].apply(get_journal_name_original)
works["primary_location"] = works["primary_location"].apply(remove_parentheses)
works["primary_topic"] = works["primary_topic"].apply(get_topic, class_="field")

In [26]:
from collections import Counter
Counter(works["primary_location"])

Counter({'Science communication': 35,
         'Scientometrics': 77,
         'Research evaluation': 4,
         'Science, technology & society': 13,
         'Public understanding of science': 35,
         'Social studies of science': 4,
         'JCOM, journal of science communication': 5,
         'Science, technology & human values/Science, technology, & human values': 5,
         'Research policy': 12,
         'East Asian science, technology and society': 13,
         'Science as culture': 5})

In [27]:
import os
os.chdir("/content/drive/MyDrive/卒論")

In [28]:
works.to_json("PCA.json", orient = "records")